# COVID19 - District Region of Sao Paulo State, Brazil

In [1]:
from environs import Env
env = Env()
env.str("CUDA_DEVICE_ORDER",'PCI_BUS_ID')
env.str("CUDA_VISIBLE_DEVICES",'0,1,2,3,4')
env.int("NUMBA_ENABLE_CUDASIM",1)
env.bool("OMPI_MCA_opal_cuda_support",True)

import os
import ray
# MB=1024*1024
# GB=MB*1024
# ray.init(num_gpus=5,num_cpus=1)
# @ray.remote(num_gpus=1)
# def use_gpu():
#     print("ray.get_gpu_ids(): {}".format(ray.get_gpu_ids()[0]))
#     print("CUDA_VISIBLE_DEVICES: {}".format(os.environ["CUDA_VISIBLE_DEVICES"]))

# use_gpu.remote()


ray.shutdown()
ray.init()

2021-02-26 10:03:16,165	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '192.168.0.104',
 'raylet_ip_address': '192.168.0.104',
 'redis_address': '192.168.0.104:13327',
 'object_store_address': '/tmp/ray/session_2021-02-26_10-03-15_600673_18404/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-02-26_10-03-15_600673_18404/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2021-02-26_10-03-15_600673_18404',
 'metrics_export_port': 57811,
 'node_id': '86ddc4fa8a65912263ea16fa9510a1cd3cc1bf3b'}

In [2]:
import urllib.request
import pandas as pd
import numpy as np
from datetime import datetime,timedelta,date

In [3]:
# Download data
import get_data
LoadData=False

if LoadData:
    get_data.get_data()

In [4]:
dfSP = pd.read_csv("data/dados_municipios_SP.csv.gz",compression='gzip')
query=dfSP.query('DRS == "{}"'.format('DRS 01 - Grande São Paulo'))
query2=query[query['date']=="2020-12-27"]
display(query2)
display(query2['popEst'].sum())

,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,popEst,estimated_population,city_ibge_code,confirmed_per_100k_inhabitants,death_rate,DRS
10737,2020-12-27,SP,Arujá,city,2812,111,262,False,89824.0,91157.0,3503901.0,3084.78778,0.0395,DRS 01 - Grande São Paulo
15875,2020-12-27,SP,Barueri,city,11200,440,266,False,274182.0,276982.0,3505708.0,4043.58406,0.0393,DRS 01 - Grande São Paulo
19039,2020-12-27,SP,Biritiba Mirim,city,557,33,236,False,32598.0,32936.0,3506607.0,1691.15861,0.0592,DRS 01 - Grande São Paulo
26483,2020-12-27,SP,Caieiras,city,3258,109,263,False,101470.0,102775.0,3509007.0,3170.03162,0.0335,DRS 01 - Grande São Paulo
27046,2020-12-27,SP,Cajamar,city,1850,84,260,False,76801.0,77934.0,3509205.0,2373.80347,0.0454,DRS 01 - Grande São Paulo
31606,2020-12-27,SP,Carapicuíba,city,11966,435,268,False,400927.0,403183.0,3510609.0,2967.88307,0.0364,DRS 01 - Grande São Paulo
38285,2020-12-27,SP,Cotia,city,5679,230,266,False,249210.0,253608.0,3513009.0,2239.28267,0.0405,DRS 01 - Grande São Paulo
41276,2020-12-27,SP,Diadema,city,11367,479,261,False,423884.0,426757.0,3513801.0,2663.57670,0.0421,DRS 01 - Grande São Paulo
45437,2020-12-27,SP,Embu das Artes,city,4485,208,263,False,273726.0,276535.0,3515004.0,1621.85618,0.0464,DRS 01 - Grande São Paulo
45753,2020-12-27,SP,Embu-Guaçu,city,1710,58,253,False,69385.0,69901.0,3515103.0,2446.31693,0.0339,DRS 01 - Grande São Paulo


21734682.0

# Model

In [5]:
# lista DRSs
DRS = list(dfSP["DRS"].unique())
DRS.remove("Indefinido")
DRS

['DRS 09 - Marília',
 'DRS 15 - São José do Rio Preto',
 'DRS 14 - São João da Boa Vista',
 'DRS 06 - Bauru',
 'DRS 16 - Sorocaba',
 'DRS 11 - Presidente Prudente',
 'DRS 05 - Barretos',
 'DRS 13 - Ribeirão Preto',
 'DRS 02 - Araçatuba',
 'DRS 07 - Campinas',
 'DRS 03 - Araraquara',
 'DRS 10 - Piracicaba',
 'DRS 17 - Taubaté',
 'DRS 08 - Franca',
 'DRS 01 - Grande São Paulo',
 'DRS 12 - Registro',
 'DRS 04 - Baixada Santista']

# SEAIR-D Model Equations

$$\Large \begin{array}{l}
\dfrac{d s}{d t}=-[ i(t) + \beta_R \cdot a(t)] \cdot \beta \cdot s(t)\\ 
\dfrac{d e}{d t}=[i(t) + \beta_R \cdot a(t)] \cdot \beta \cdot s(t) -\sigma \cdot e(t) - \mu \cdot e(t)\\ 
\dfrac{d a}{d t}=\sigma \cdot e(t) \cdot (1-p)-\gamma \cdot a(t)  - \mu \cdot a(t)\\
\dfrac{d i}{d t}=\sigma \cdot e(t) \cdot p - \gamma \cdot i(t)  - \mu \cdot i(t)\\ 
\dfrac{d r}{d t}=b_\gamma \cdot i(t) + \gamma \cdot a(t) + \nu \cdot k(t)\\
\dfrac{d k}{d t}=(a_\gamma + \nu ) \cdot k(t) + \mu \cdot [e(t)+a(t)+i(t)]
\end{array}$$

The recovery and death rates are calculated from $\gamma$

$$\Large \gamma=a_\gamma+b_\gamma$$

The sigmoid function:

$$\Large S(t)=\frac{1}{1 + e^{-t}}$$

The $\sigma$ variation is modeled as

$$\Large 
    \sigma = \sigma_i \cdot [1-S(t)]+\sigma_f \cdot S(t)
$$

See below the transition between two value - $\sigma_i$ and $\sigma_f$ -  based on Sigmoids Curves when $\sigma_i$<$\sigma_f$, which is indicated by the blue curve, and $\sigma_i$>$\sigma_f$, which is indicated by the red curve.

![Sigmoids Curves when $\sigma_i$<$\sigma_f$ and $\sigma_i$>$\sigma_f$](../countries/results/sigmoid.png)
<h4><center>Sigmoids Curves when $\sigma_i$<$\sigma_f$ and $\sigma_i$>$\sigma_f$</center></h4>


The $\sigma$ with sigmoid is performed two times inside the data interval, so we have an initial value constant of $\sigma_i$, a sigmoid transition to value $\sigma_f$, a period of $\sigma_f$ constant, after a sigmoid transition to value $\sigma_{ff}$ and a period of $\sigma_{ff}$ constant. The times where the two sigmoids are placed are calculated by optimization as well as the values of $\sigma_i$,$\sigma_f$ and $\sigma_{ff}$.

The sum of all people accounted is equal to population considered:

$$\Large s + e + a + i + r + k = N$$

The derivate is:

$$\Large \frac{d s}{d t}+\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t}+\frac{d k}{d t} = 0$$

The last equation does not need to be solve because:

$$\Large \frac{d k}{d t}=-(\frac{d s}{d t}+\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t})$$

The sum of all rates are equal to zero! The importance of this equation is that it conservates the rates.


## Parameters

### Fixed parameters:

N: the population considered in the study

$p$: is the fraction of the exposed which become symptomatic infectious sub-population.

$(1-p)$: is the fraction of the exposed which becomes asymptomatic infectious sub-population.

### Optimized Parameters

All parameters below are calculated through evolutionary optimization package Yabox to fit real data:

$\beta$: Effective contact rate [1/day]

$\beta_R$: Fraction of effective contact rate [1/day]
    
$\gamma$: Recovery(+Mortality) rate $\gamma=(a_\gamma+b_\gamma)$ [1/day]

$a_\gamma$: mortality of infected  [1/day]

$b_\gamma$: recovery rate  [1/day]

$\sigma$: is the rate at which individuals move from the exposed to the infectious classes. Three parameters $\sigma_i$, $\sigma_f$ and $\sigma_{ff}$ are used along the days. The transition from $\sigma_i$ to $\sigma_f$ and from $\sigma_f$ to $\sigma_{ff}$ are made by a sigmoid function. Its reciprocal  ($1/\sigma$) is the average latent (exposed) period. [1/day]

$\nu$: medicine and hospital improvement rate [1/day] 

$\mu$: direct mortality rate [1/day] 

## Options: Param.CSV file, changeCSV, Adjust Param, Save CSV, Select Specific DRS

In [6]:
from IPython.display import display, HTML, Markdown

paramOpt=2
changeCSV=True
adjustParam=False
paramSave=False
selectDRS=False

## Different Options to Load Param.CSV file

In [7]:
if paramOpt==-1:
    paramFile="data/param.csv"
    version = "-1"
    model = "ManualIC"
    
if paramOpt==0:
    paramFile= "data/param_optimized_Yabox_HistMin.csv"
    version = "0"
    model = "LastIC"
    
if paramOpt==1:
    version = "116"
    model = "YaboxIC"    
    paramFile="data/param_optimized_Yabox_HistMin"+version+".csv"

if paramOpt==2:
    version = "119"
    model = "YaboxIC"    
    paramFile="data/param_optimized_Yabox_HistMin"+version+".csv"


In [8]:
dfparam = pd.read_csv(paramFile)
dfparam = dfparam.dropna()
dfparam.e0=0
dfparam.r0=0

display(Markdown("## Original Initial Parameters"))
display(dfparam)

## Original Initial Parameters

,DRS,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,DRS 01 - Grande São Paulo,2020-03-21,200,20268871,0,0,41,0,233,140,0.15,0.3070,0.0
1,DRS 02 - Araçatuba,2020-03-30,200,660592,0,0,234,0,71,6,0.15,0.2136,0.0
2,DRS 03 - Araraquara,2020-04-02,200,711092,0,0,121,0,151,73,0.15,0.2617,0.0
3,DRS 04 - Baixada Santista,2020-03-28,200,810677,0,0,184,0,161,45,0.15,0.4406,0.0
4,DRS 05 - Barretos,2020-04-04,200,295706,0,0,164,0,215,182,0.15,0.3075,0.0
5,DRS 06 - Bauru,2020-04-04,200,821721,0,0,210,0,246,191,0.15,0.2608,0.0
6,DRS 07 - Campinas,2020-03-28,200,2091996,0,0,109,0,249,130,0.15,0.6601,0.0
7,DRS 08 - Franca,2020-04-02,200,562529,0,0,101,0,13,40,0.15,0.8251,0.0
8,DRS 09 - Marília,2020-04-04,200,744101,0,0,150,0,244,177,0.15,0.3622,0.0
9,DRS 10 - Piracicaba,2020-03-28,200,1078979,0,0,101,0,195,191,0.15,0.2597,0.0


## Modify param.csv file to fit better the data

In [9]:
#     dfparam['s0'] = pd.to_numeric(dfparam['s0'],errors='coerce')
#     dfparam['i0'] = pd.to_numeric(dfparam['i0'],errors='coerce')
#     dfparam['d0'] = pd.to_numeric(dfparam['d0'],errors='coerce')
#     dfparam['s0'] = dfparam['s0'].astype(float)
#     dfparam['i0'] = dfparam['i0'].astype(float)
#     dfparam['d0'] = dfparam['d0'].astype(float)
#     dfparam.loc[dfparam.DRS=='DRS 14 - São João da Boa Vista','s0'] = dfparam.loc[dfparam.DRS=='DRS 14 - São João da Boa Vista','s0'].values[0]*1.1
#     dfparam.loc[dfparam.DRS=='DRS 14 - São João da Boa Vista','i0'] = dfparam.loc[dfparam.DRS=='DRS 14 - São João da Boa Vista','i0']*.1

if changeCSV and (paramOpt==1):
    dfparam.loc[dfparam.DRS=='DRS 12 - Registro','s0'] = dfparam.loc[dfparam.DRS=='DRS 12 - Registro','s0']*2
    dfparam.loc[dfparam.DRS=='DRS 12 - Registro','d0'] = dfparam.loc[dfparam.DRS=='DRS 12 - Registro','d0']/10
    
if adjustParam:
    sCorrect=[1.38,1.0,1.05,1.5,1.5,1.4,1.1,1.5,1.15,1.55,1.35,1.2,1.35,1.15,1.55,1.45,1.25]
    dfparam.s0=dfparam.s0.multiply(sCorrect[:len(DRS)], axis=0)

if selectDRS and paramOpt==1:
    DRS=['DRS 09 - Marília','DRS 10 - Piracicaba','DRS 14 - São João da Boa Vista']

if not selectDRS:
    DRS=dfparam.DRS    

dfparam['s0'] = dfparam['s0'].astype(int)
dfparam['i0'] = dfparam['i0'].astype(int)
dfparam['d0'] = dfparam['d0'].astype(int)
dfparam['r0'] = dfparam['r0'].astype(int)

if paramSave:
    dfparam.to_csv("new_"+paramFile)    
    
display(Markdown("## Modified Initial Parameters"))

# dfparam['WREC']=dfparam['WREC'].apply(lambda x: min(x,0.001))
dfparam['WDTH']=1-dfparam['WCASES']-dfparam['WREC']

finalDate = date.today() + timedelta(days=-1)
finalDateStr = datetime.strftime(finalDate, "%Y-%m-%d")
dfparam['endDate']=finalDateStr

#poe RATIO no final
l=[0,1,2,3,4,5,6,7,8,9,11,12, 13, 14, 10] # index order
dfparam=dfparam[[dfparam.columns[i] for i in l]]

# for i in DRS:
#     query=dfSP.query('DRS == "{}"'.format(i))
#     query2=query[query['date']=="2020-12-28"]
#     dfparam.loc[dfparam['DRS']==i,'s0']=int(query2['popEst'].sum()*0.7)
    
display(dfparam)


## Modified Initial Parameters

,DRS,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,WCASES,WREC,WDTH,endDate,RATIO
0,DRS 01 - Grande São Paulo,2020-03-21,200,20268871,0,0,41,0,233,140,0.3070,0.0,0.6930,2021-02-25,0.15
1,DRS 02 - Araçatuba,2020-03-30,200,660592,0,0,234,0,71,6,0.2136,0.0,0.7864,2021-02-25,0.15
2,DRS 03 - Araraquara,2020-04-02,200,711092,0,0,121,0,151,73,0.2617,0.0,0.7383,2021-02-25,0.15
3,DRS 04 - Baixada Santista,2020-03-28,200,810677,0,0,184,0,161,45,0.4406,0.0,0.5594,2021-02-25,0.15
4,DRS 05 - Barretos,2020-04-04,200,295706,0,0,164,0,215,182,0.3075,0.0,0.6925,2021-02-25,0.15
5,DRS 06 - Bauru,2020-04-04,200,821721,0,0,210,0,246,191,0.2608,0.0,0.7392,2021-02-25,0.15
6,DRS 07 - Campinas,2020-03-28,200,2091996,0,0,109,0,249,130,0.6601,0.0,0.3399,2021-02-25,0.15
7,DRS 08 - Franca,2020-04-02,200,562529,0,0,101,0,13,40,0.8251,0.0,0.1749,2021-02-25,0.15
8,DRS 09 - Marília,2020-04-04,200,744101,0,0,150,0,244,177,0.3622,0.0,0.6378,2021-02-25,0.15
9,DRS 10 - Piracicaba,2020-03-28,200,1078979,0,0,101,0,195,191,0.2597,0.0,0.7403,2021-02-25,0.15


## Solver Loading

In [14]:
%reload_ext autoreload
%autoreload 2
import LearnerYabox_v5NewModel as Learner #Yabox

2021-02-26 10:08:37,913	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8266


## All Districts and Select District to Run

In [15]:
allDistricts=True
cleanRecovered=False

if allDistricts:
    display(DRS)
else:
    districtRegionSelected="DRS 01 - Grande São Paulo"
    display(districtRegionSelected)

0          DRS 01 - Grande São Paulo
1                 DRS 02 - Araçatuba
2                DRS 03 - Araraquara
3          DRS 04 - Baixada Santista
4                  DRS 05 - Barretos
5                     DRS 06 - Bauru
6                  DRS 07 - Campinas
7                    DRS 08 - Franca
8                   DRS 09 - Marília
9                DRS 10 - Piracicaba
10      DRS 11 - Presidente Prudente
11                 DRS 12 - Registro
12           DRS 13 - Ribeirão Preto
13    DRS 14 - São João da Boa Vista
14    DRS 15 - São José do Rio Preto
15                 DRS 16 - Sorocaba
16                  DRS 17 - Taubaté
Name: DRS, dtype: object

In [16]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return true;}

<IPython.core.display.Javascript object>

# Main Loop

In [ ]:
if allDistricts:

    if selectDRS:
        loop=[0,len(DRS)]        
    else:
        loop=[0,6,12,17]
    
    for i in range(0,len(loop)-1):
        results=[]
        for districtRegion in DRS[loop[i]:loop[i+1]]:
            query = dfparam.query('DRS == "{}"'.format(districtRegion)).reset_index()
            parameters = np.array(query.iloc[:, 2:])[0]
            dR=ray.put(districtRegion)
            cR=ray.put(cleanRecovered)
            vR=ray.put(version)
            f=Learner.Learner.remote(dR, *parameters, cR, vR)
            result = f.train.remote() 
            results.append(result)
        display("Running DRS {:d} to {:d}".format(loop[i]+1,loop[i+1]))
        display(DRS[loop[i]:loop[i+1]])
        resultsFinal = ray.get(results)

else:
    results=[]
    districtRegion= districtRegionSelected 
    query = dfparam.query('DRS == "{}"'.format(districtRegion)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    dR=ray.put(districtRegion)
    cR=ray.put(cleanRecovered)
    vR=ray.put(version)
    f=Learner.Learner.remote(dR, *parameters, cR, vR)
    result = f.train.remote() 
    results.append(result)
    display(districtRegion)
    resultsFinal = ray.get(results)


'Running DRS 1 to 6'

0    DRS 01 - Grande São Paulo
1           DRS 02 - Araçatuba
2          DRS 03 - Araraquara
3    DRS 04 - Baixada Santista
4            DRS 05 - Barretos
5               DRS 06 - Bauru
Name: DRS, dtype: object

  0%|          | 0/10125000.0 [00:00<?, ?it/s]
  0%|          | 0/10125000.0 [00:00<?, ?it/s]
  0%|          | 0/10125000.0 [00:00<?, ?it/s]
  0%|          | 0/10125000.0 [00:00<?, ?it/s]
  0%|          | 0/10125000.0 [00:00<?, ?it/s]
  0%|          | 0/10125000.0 [00:00<?, ?it/s]
  0%|          | 0/10125000.0 [00:01<?, ?it/s]
  0%|          | 0/10125000.0 [00:01<?, ?it/s]
  0%|          | 0/10125000.0 [00:01<?, ?it/s]
  0%|          | 0/10125000.0 [00:01<?, ?it/s]
  0%|          | 0/10125000.0 [00:01<?, ?it/s]
  0%|          | 0/10125000.0 [00:01<?, ?it/s]
  0%|          | 1/10125000.0 [00:01<1159:55:35,  2.42it/s]
  0%|          | 1/10125000.0 [00:01<1185:57:21,  2.37it/s]
  0%|          | 1/10125000.0 [00:01<1167:26:58,  2.41it/s]
  0%|          | 1/10125000.0 [00:01<1165:51:07,  2.41it/s]
  0%|          | 1/10125000.0 [00:01<1179:08:50,  2.39it/s]
  0%|          | 3/10125000.0 [00:01<989:08:13,  2.84it/s] 
  0%|          | 1/10125000.0 [00:01<1496:23:34,  1.88it/s]
  0%|          |

  0%|          | 253/10125000.0 [00:11<70:01:16, 40.17it/s]
  0%|          | 153/10125000.0 [00:11<124:33:15, 22.58it/s]
  0%|          | 253/10125000.0 [00:11<69:57:49, 40.20it/s]
  0%|          | 276/10125000.0 [00:11<64:21:34, 43.70it/s]
  0%|          | 253/10125000.0 [00:11<72:41:24, 38.69it/s]
  0%|          | 253/10125000.0 [00:11<70:35:35, 39.84it/s]
  0%|          | 276/10125000.0 [00:11<65:42:21, 42.80it/s]
  0%|          | 171/10125000.0 [00:11<114:58:51, 24.46it/s]
  0%|          | 276/10125000.0 [00:11<66:26:13, 42.33it/s]
  0%|          | 300/10125000.0 [00:11<61:16:01, 45.90it/s]
  0%|          | 276/10125000.0 [00:12<68:42:07, 40.94it/s]
  0%|          | 276/10125000.0 [00:12<66:57:57, 42.00it/s]
  0%|          | 300/10125000.0 [00:12<62:30:44, 44.99it/s]
  0%|          | 300/10125000.0 [00:12<63:20:12, 44.40it/s]
  0%|          | 190/10125000.0 [00:12<106:28:08, 26.42it/s]
  0%|          | 325/10125000.0 [00:12<59:24:23, 47.34it/s]
  0%|          | 300/10125000.0 [00:1

  0%|          | 1035/10125000.0 [00:23<34:13:15, 82.18it/s]
  0%|          | 1081/10125000.0 [00:23<33:38:13, 83.60it/s]
  0%|          | 1035/10125000.0 [00:23<35:03:37, 80.21it/s]
  0%|          | 703/10125000.0 [00:23<51:02:32, 55.10it/s]
  0%|          | 1081/10125000.0 [00:23<34:00:13, 82.70it/s]
  0%|          | 1176/10125000.0 [00:23<30:58:45, 90.78it/s]
  0%|          | 1081/10125000.0 [00:23<33:28:08, 84.02it/s]
  0%|          | 1128/10125000.0 [00:23<32:30:38, 86.50it/s]
  0%|          | 1081/10125000.0 [00:23<33:56:26, 82.86it/s]
  0%|          | 1128/10125000.0 [00:24<33:19:13, 84.40it/s]
  0%|          | 1225/10125000.0 [00:23<30:17:18, 92.85it/s]
  0%|          | 1128/10125000.0 [00:24<32:39:18, 86.12it/s]
  0%|          | 1176/10125000.0 [00:24<31:34:33, 89.06it/s]
  0%|          | 741/10125000.0 [00:24<49:39:47, 56.63it/s]
  0%|          | 1128/10125000.0 [00:24<33:19:22, 84.39it/s]
  0%|          | 1275/10125000.0 [00:24<29:47:55, 94.37it/s]
  0%|          | 1176/1012

...

  0%|          | 9453/10125000.0 [01:20<15:01:37, 186.99it/s]
  0%|          | 10296/10125000.0 [01:20<12:03:35, 232.97it/s]
  0%|          | 8911/10125000.0 [01:21<15:16:33, 183.95it/s]
  0%|          | 9730/10125000.0 [01:21<12:41:55, 221.26it/s]
  0%|          | 9453/10125000.0 [01:21<14:13:27, 197.54it/s]
  0%|          | 10440/10125000.0 [01:21<11:59:22, 234.34it/s]
  0%|          | 9591/10125000.0 [01:21<15:13:58, 184.46it/s]
  0%|          | 6903/10125000.0 [01:21<20:48:17, 135.09it/s]
  0%|          | 9870/10125000.0 [01:21<12:48:33, 219.35it/s]
  0%|          | 9045/10125000.0 [01:22<15:11:00, 185.07it/s]
  0%|          | 10585/10125000.0 [01:22<11:52:44, 236.51it/s]
  0%|          | 9591/10125000.0 [01:22<14:34:22, 192.81it/s]
  0%|          | 9730/10125000.0 [01:22<14:41:58, 191.15it/s]
  0%|          | 7021/10125000.0 [01:22<20:58:26, 134.00it/s]
  0%|          | 10011/10125000.0 [01:22<12:27:58, 225.39it/s]
  0%|          | 9180/10125000.0 [01:22<15:02:36, 186.79it/s]
  0%

  0%|          | 11935/10125000.0 [01:36<13:14:32, 212.14it/s]
  0%|          | 13366/10125000.0 [01:36<10:40:20, 263.18it/s]
  0%|          | 12720/10125000.0 [01:36<11:47:30, 238.21it/s]
  0%|          | 14365/10125000.0 [01:36<9:47:48, 286.67it/s]
  0%|          | 12561/10125000.0 [01:36<13:17:21, 211.37it/s]
  0%|          | 13530/10125000.0 [01:36<10:41:00, 262.90it/s]
  0%|          | 12880/10125000.0 [01:36<11:42:50, 239.79it/s]
  0%|          | 12090/10125000.0 [01:36<13:01:41, 215.62it/s]
  0%|          | 9045/10125000.0 [01:36<19:05:42, 147.16it/s]
  0%|          | 14535/10125000.0 [01:37<9:45:09, 287.97it/s]
  0%|          | 12720/10125000.0 [01:37<13:06:18, 214.34it/s]
  0%|          | 13695/10125000.0 [01:37<10:34:42, 265.51it/s]
  0%|          | 13041/10125000.0 [01:37<11:33:23, 243.05it/s]
  0%|          | 12246/10125000.0 [01:37<13:04:36, 214.82it/s]
  0%|          | 14706/10125000.0 [01:37<9:50:02, 285.58it/s]
  0%|          | 9180/10125000.0 [01:37<18:47:26, 149.54it/

  0%|          | 16653/10125000.0 [01:51<10:18:53, 272.21it/s]
  0%|          | 15931/10125000.0 [01:51<11:41:41, 240.11it/s]
  0%|          | 11325/10125000.0 [01:51<17:08:15, 163.93it/s]
  0%|          | 17766/10125000.0 [01:51<9:26:01, 297.61it/s]
  0%|          | 19110/10125000.0 [01:51<8:28:40, 331.12it/s]
  0%|          | 15400/10125000.0 [01:51<12:15:59, 228.93it/s]
  0%|          | 16836/10125000.0 [01:52<10:12:54, 274.87it/s]
  0%|          | 16110/10125000.0 [01:52<11:36:58, 241.73it/s]
  0%|          | 17955/10125000.0 [01:52<9:22:12, 299.62it/s]
  0%|          | 19306/10125000.0 [01:52<8:22:16, 335.33it/s]
  0%|          | 11476/10125000.0 [01:52<17:15:26, 162.79it/s]
  0%|          | 17020/10125000.0 [01:52<9:58:56, 281.27it/s] 
  0%|          | 15576/10125000.0 [01:52<12:07:54, 231.47it/s]
  0%|          | 16290/10125000.0 [01:52<11:25:30, 245.77it/s]
  0%|          | 18145/10125000.0 [01:52<9:24:52, 298.21it/s]
  0%|          | 19503/10125000.0 [01:53<8:22:43, 335.03it/s

  0%|          | 21321/10125000.0 [02:06<8:35:46, 326.48it/s]
  0%|          | 19110/10125000.0 [02:06<10:40:18, 263.05it/s]
  0%|          | 22578/10125000.0 [02:06<8:15:46, 339.61it/s]
  0%|          | 20100/10125000.0 [02:07<9:59:53, 280.74it/s] 
  0%|          | 24310/10125000.0 [02:07<7:54:24, 354.86it/s]
  0%|          | 14028/10125000.0 [02:07<15:29:15, 181.34it/s]
  0%|          | 21528/10125000.0 [02:07<8:42:22, 322.36it/s]
  0%|          | 19306/10125000.0 [02:07<10:39:45, 263.27it/s]
  0%|          | 22791/10125000.0 [02:07<8:27:48, 331.56it/s]
  0%|          | 24531/10125000.0 [02:07<8:07:36, 345.24it/s]
  0%|          | 21736/10125000.0 [02:07<8:42:00, 322.58it/s]
  0%|          | 20301/10125000.0 [02:07<10:14:53, 273.89it/s]
  0%|          | 23005/10125000.0 [02:08<8:47:50, 318.98it/s]
  0%|          | 19503/10125000.0 [02:08<10:48:07, 259.86it/s]
  0%|          | 14196/10125000.0 [02:08<15:56:20, 176.21it/s]
  0%|          | 24753/10125000.0 [02:08<8:06:50, 345.77it/s]
 

  0%|          | 24531/10125000.0 [02:22<9:08:06, 307.13it/s]
  0%|          | 30135/10125000.0 [02:22<7:01:15, 399.40it/s]
  0%|          | 26796/10125000.0 [02:22<7:56:41, 353.07it/s]
  0%|          | 17020/10125000.0 [02:22<13:54:59, 201.76it/s]
  0%|          | 28203/10125000.0 [02:22<7:32:30, 371.88it/s]
  0%|          | 23653/10125000.0 [02:22<9:29:31, 295.61it/s]
  0%|          | 24753/10125000.0 [02:22<9:08:49, 306.73it/s]
  0%|          | 30381/10125000.0 [02:23<6:58:30, 402.01it/s]
  0%|          | 27028/10125000.0 [02:23<7:49:31, 358.44it/s]
  0%|          | 28441/10125000.0 [02:23<7:31:06, 373.03it/s]
  0%|          | 23871/10125000.0 [02:23<9:25:43, 297.59it/s]
  0%|          | 17205/10125000.0 [02:23<13:36:45, 206.26it/s]
  0%|          | 24976/10125000.0 [02:23<9:09:18, 306.45it/s]
  0%|          | 30628/10125000.0 [02:23<6:57:08, 403.32it/s]
  0%|          | 27261/10125000.0 [02:23<7:51:10, 357.19it/s]
  0%|          | 28680/10125000.0 [02:23<7:30:47, 373.27it/s]
  0%| 

In [ ]:
ray.shutdown()

# Save Results in CSV format 

In [ ]:
# Save data as csv
import glob
import os

path = './results/data'
files = glob.glob(os.path.join(path, "*"+version+".csv"))
display(files)

df = (pd.read_csv(f).assign(DRS = f.split(" - ")[-1].split(version)[0]) for f in files)
df_all_drs = pd.concat(df, ignore_index=True)
df_all_drs.index.name = 'index'
df_all_drs.to_csv('./data/SEAIRD_sigmaOpt_AllDRS'+version+'.csv', sep=",")